In [2]:
import os

In [5]:
%pwd

'c:\\Users\\prasa\\OneDrive\\Desktop\\DSA hackathons\\Text summarizer for studiying\\Text-summarizer-cicd'

In [4]:
os.chdir("../")

In [11]:
from dataclasses import dataclass
from pathlib import Path 
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_URL: str 
    local_data_file: Path 
    unzip_dir: Path 
    
    

In [82]:

import os
from box.exceptions import BoxValueError
import yaml
from textSummarizer.logging import logger
from ensure import ensure_annotations
from box import ConfigBox
from pathlib import Path
from typing import Any



@ensure_annotations
def read_yaml(path_to_yaml: Path) -> ConfigBox:
    """reads yaml file and returns

    Args:
        path_to_yaml (str): path like input

    Raises:
        ValueError: if yaml file is empty
        e: empty file

    Returns:
        ConfigBox: ConfigBox type
    """
    try:
        with open(path_to_yaml) as yaml_file:
            content = yaml.safe_load(yaml_file)
            logger.info(f"yaml file: {path_to_yaml} loaded successfully")
            return ConfigBox(content)
    except BoxValueError:
        raise ValueError("yaml file is empty")
    except Exception as e:
        raise e
    


@ensure_annotations
def create_directories(path_to_directories: list, verbose=True):
    """create list of directories

    Args:
        path_to_directories (list): list of path of directories
        ignore_log (bool, optional): ignore if multiple dirs is to be created. Defaults to False.
    """
    for path in path_to_directories:
        os.makedirs(path, exist_ok=True)
        if verbose:
            logger.info(f"created directory at: {path}")



@ensure_annotations
def get_size(path: Path) -> str:
    """get size in KB

    Args:
        path (Path): path of the file

    Returns:
        str: size in KB
    """
    size_in_kb = round(os.path.getsize(path)/1024)
    return f"~ {size_in_kb} KB"

    

    


TypeError: read_yaml() missing 1 required positional argument: 'path_to_yaml'

In [47]:
from textSummarizer.constants import*

In [90]:
CONFIG_FILE_PATH=Path(r"C:\Users\prasa\OneDrive\Desktop\DSA hackathons\Text summarizer for studiying\Text-summarizer-cicd\config\config.yaml")
PARAMS_FILE_PATH=Path(r"C:\Users\prasa\OneDrive\Desktop\DSA hackathons\Text summarizer for studiying\Text-summarizer-cicd\params.yaml")

In [ ]:
path_to_directories = ["path/to/directory1", "path/to/directory2"]
create_directories(path_to_directories)


In [94]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        #list(self.config.artifacts_root)
        create_directories(self.config.artifacts_root)
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config 
    

In [103]:
import os
import urllib.request as request
import urllib 
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import*
#request=urllib.urlretrieve()

In [57]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [98]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-30 10:53:36,793: INFO: 2782862417: yaml file: C:\Users\prasa\OneDrive\Desktop\DSA hackathons\Text summarizer for studiying\Text-summarizer-cicd\config\config.yaml loaded successfully]
[2024-03-30 10:53:36,796: INFO: 2782862417: yaml file: C:\Users\prasa\OneDrive\Desktop\DSA hackathons\Text summarizer for studiying\Text-summarizer-cicd\params.yaml loaded successfully]
[2024-03-30 10:53:36,799: INFO: 2782862417: created directory at: artifacts]
[2024-03-30 10:53:36,802: INFO: 2782862417: created directory at: artifacts/data_ingestion]


NameError: name 'request' is not defined

In [97]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        # Ensure artifacts_root is treated as a list
        artifacts_root = self.config.artifacts_root
        if not isinstance(artifacts_root, list):
            artifacts_root = [artifacts_root]
        
        # Call create_directories with the list of artifacts_root
        create_directories(artifacts_root)
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        # Ensure root_dir is treated as a list
        root_dir = config.root_dir
        if not isinstance(root_dir, list):
            root_dir = [root_dir]
        
        # Call create_directories with the list of root_dir
        create_directories(root_dir)
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config 


In [105]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-30 10:57:33,704: INFO: 2782862417: yaml file: C:\Users\prasa\OneDrive\Desktop\DSA hackathons\Text summarizer for studiying\Text-summarizer-cicd\config\config.yaml loaded successfully]
[2024-03-30 10:57:33,707: INFO: 2782862417: yaml file: C:\Users\prasa\OneDrive\Desktop\DSA hackathons\Text summarizer for studiying\Text-summarizer-cicd\params.yaml loaded successfully]
[2024-03-30 10:57:33,711: INFO: 2782862417: created directory at: artifacts]
[2024-03-30 10:57:33,713: INFO: 2782862417: created directory at: artifacts/data_ingestion]


HTTPError: HTTP Error 404: Not Found